# Twiter Sentiment Analysis
![Twitter Sentiment Analysis](https://miro.medium.com/max/644/1*OAR9kjMonh8rZJj0en9evQ.png)

**Sentiment Analysis** also known as “Opinion Mining”, Sentiment Analysis refers to the use of Natural Language Processing to determine the attitude, opinions and emotions of a speaker, writer, or other subject.

> It is the process of determining whether a piece of text or sentence is positive or negative. This is also called the *Polarity of the content*.

**Problem Statement**

A sentiment analysis job about the problems of each major U.S. airline and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as “late flight” or “rude service”).

**DataSet**

This data originally came from [Crowdflower's Data for Everyone library.](http://www.crowdflower.com/data-for-everyone)
Data contains whether the sentiment of the tweets in this set was positive, neutral, or negative for six US airlines:

1. American
2. Delta
3. SouthWest
4. United
5. US Airways
6. Virgin America

In [ ]:
# Importing Libraries 

from fastai.metrics import accuracy
from fastai.text import *
from fastai import *
import gc
import os
import pandas as pd
import numpy as np

We will be using UMLFiT model implemented in fast.Ai for our task.

**What is ULMFiT ?**

> **ULMFiT** is essentially a method to enable transfer learning for any NLP task and achieve great results. All this, without having to train models from scratch. 

For more in details, please refer [ULMFiT Paper](https://arxiv.org/abs/1801.06146)

In [ ]:
tweets = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')
print('Shape: ', tweets.shape)
tweets.head()

In [ ]:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum()
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return(np.transpose(output))
    else:
        return(False)

In [ ]:
check_missing_data(tweets)

In [ ]:
# droping 3 columns which are 90% empty

tweets = tweets.drop(columns=['airline_sentiment_gold', 'negativereason_gold', 'tweet_coord'])

### Data Visualization

In [ ]:
tweets.groupby('airline')['airline_sentiment'].count().sort_values(ascending=False).plot(kind='bar')
plt.title('Number of tweets wrt Airlines')
plt.show()

In [ ]:
airlines= ['US Airways','United','American','Southwest','Delta','Virgin America']

plt.figure(1,figsize=(16, 12))
for i in airlines:
    indices= airlines.index(i)
    plt.subplot(2,3,indices+1)
    new_df=tweets[tweets['airline']==i]
    count=new_df['airline_sentiment'].value_counts()
    Index = [1,2,3]
    plt.bar(Index,count, color=['red', 'green', 'blue'])
    plt.xticks(Index,['negative','neutral','positive'])
    plt.ylabel('Mood Count')
    plt.xlabel('Mood')
    plt.title('Count of Moods of '+i)

***Wordclouds*** of the **Positive**, **Negative**, **Neutral** - airline_sentiment

In [ ]:
from wordcloud import WordCloud,STOPWORDS

def generate_wordcloud(text, title): 
    wordcloud = WordCloud(relative_scaling = 1.0,stopwords = STOPWORDS, background_color='black').generate(text)
    fig,ax = plt.subplots(1,1,figsize=(14,18))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")
    ax.margins(x=0, y=0)
    plt.title(title)
    plt.show()

In [ ]:
words_positive = ' '.join(tweets[tweets['airline_sentiment']=='positive']['text'])
cleaned_word_positive = " ".join([word for word in words_positive.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

generate_wordcloud(cleaned_word_positive, 'Positive Tweets')

In [ ]:
words_negative = ' '.join(tweets[tweets['airline_sentiment']=='negative']['text'])
cleaned_word_negative = " ".join([word for word in words_negative.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

generate_wordcloud(cleaned_word_negative, 'Negative Tweets')

In [ ]:
words_neutral = ' '.join(tweets[tweets['airline_sentiment']=='neutral']['text'])
cleaned_word_neutral = " ".join([word for word in words_neutral.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

generate_wordcloud(cleaned_word_negative, 'Neutral Tweets')

### Data Cleansing 

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

def clean_tweet_length(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return(len(meaningful_words)) 

tweets['clean_tweet']=tweets['text'].apply(lambda x: tweet_to_words(x))
tweets['Tweet_length']=tweets['text'].apply(lambda x: clean_tweet_length(x))
tweets['sentiment']=tweets['airline_sentiment'].apply(lambda x: 0 if x=='negative' else 1)

In [ ]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

# split data into training and validation set
df_trn, df_val = train_test_split(tweets, stratify = tweets['airline_sentiment'], test_size = 0.4, random_state = 678)

### ULMFiT Model

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.4)

In [ ]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# unfreeze the learner object and train
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2, 1e-5))

In [ ]:
learn.save_encoder('ft_enc')

<center><h3> **Accuracy is >80%** </h3></center>